In [15]:
import pandas as pd
import numpy as np

In [16]:
df = pd.read_csv('../data/GTspec_pdb_query.protein_structure_info.just_glyco.csv').drop_duplicates()
df = df.dropna(subset=['struct_window_angstrom_resnum'])

In [17]:
print(df.shape)
df = df[df['struct_residue'].isin(['N'])]
print(df.shape)

(3657, 35)
(3072, 35)


## N glycosylation sites may be easier to evaluate 
- S, T are O-glycosylation
- Anything that is not S, T or N should be filtered

In [ ]:
proteins = []
aas = []
distances = []
for i in df.index:
    distance_and_aa_list = df.loc[i]['angstroms_to_feature'].split('|')
    aa_list = [da.split('_')[-1] for da in distance_and_aa_list]
    distance_list = [da.split('_')[0] for da in distance_and_aa_list]
    protein = [df.loc[i]['gene_id']] * len(distance_list)
    
    aas.extend(aa_list)
    distances.extend(distance_list)
    proteins.extend(protein)

In [6]:
merged = pd.concat([pd.Series(proteins),
           pd.Series(aas,dtype=int),
           pd.Series(distances,dtype=float)],
           axis=1)

/Users/d/work/conda/lib/python3.7/site-packages/numpy/core/numeric.py:2339: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [7]:
merged.columns = ['protein','aa','distance_to_feature']

,protein,aa,distance_to_feature
0,P33527,203,85.88
1,P33527,204,78.33
2,P33527,205,81.99
3,P33527,206,76.13
4,P33527,207,77.34
5,P33527,208,69.64
6,P33527,209,70.78
7,P33527,210,65.18
8,P33527,211,72.29
9,P33527,212,71.75


## If multiple features per amino acid, keep the closest distance one

In [13]:
print(merged.shape)
merged = merged.sort_values(by='distance_to_feature').drop_duplicates(subset=['protein','aa'],keep='first')
print(merged.shape)

(1418021, 3)
(390629, 3)


In [14]:
merged.to_csv('../data/protein_aa_distance.txt',sep='\t',header=None)